In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load dataset
file_path = "WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(file_path)

print("Shape:", df.shape)
df.head()

Shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
# Basic structure check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
# Convert TotalCharges to numeric (handles blanks/spaces)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Fill missing TotalCharges: tenure==0 => 0, else median
df.loc[(df["tenure"] == 0) & (df["TotalCharges"].isna()), "TotalCharges"] = 0
df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].median())

# Check missing values after conversion
df.isna().sum().sort_values(ascending=False).head(10)

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [ ]:
# Create churn target
df["churn_flag"] = (df["Churn"].str.strip().str.lower() == "yes").astype(int)
df["churn_label"] = df["churn_flag"].map({0: "No", 1: "Yes"})

df[["Churn", "churn_flag", "churn_label"]].head()

,Churn,churn_flag,churn_label
0,No,0,No
1,No,0,No
2,Yes,1,Yes
3,No,0,No
4,Yes,1,Yes


In [ ]:
# Tenure buckets
df["tenure_group"] = pd.cut(
    df["tenure"],
    bins=[-1, 6, 12, 24, 48, 60, 1000],
    labels=["0-6", "7-12", "13-24", "25-48", "49-60", "60+"]
)

# Monthly charges quartiles
df["monthly_charges_group"] = pd.qcut(
    df["MonthlyCharges"], q=4, labels=["Low", "Medium", "High", "Very High"]
)

# Value segment quartiles (TotalCharges)
df["value_segment"] = pd.qcut(
    df["TotalCharges"], q=4, labels=["Low Value", "Mid-Low", "Mid-High", "High Value"]
)

df[["tenure_group","monthly_charges_group","value_segment"]].head()

,tenure_group,monthly_charges_group,value_segment
0,0-6,Low,Low Value
1,25-48,Medium,Mid-High
2,0-6,Medium,Low Value
3,25-48,Medium,Mid-High
4,0-6,High,Low Value


In [ ]:
# Business metrics for Visulization
df["revenue_at_risk_12m"] = df["MonthlyCharges"] * 12 * df["churn_flag"]  # simple 12-month loss assumption
df["high_value_customer"] = (df["value_segment"] == "High Value").astype(int)

# A simple ARPU proxy per row (MonthlyCharges)
df["arpu_proxy"] = df["MonthlyCharges"]

df[["revenue_at_risk_12m","high_value_customer","arpu_proxy"]].head()

,revenue_at_risk_12m,high_value_customer,arpu_proxy
0,0.0,0,29.85
1,0.0,0,56.95
2,646.2,0,53.85
3,0.0,0,42.30
4,848.4,0,70.70


In [ ]:
# KPI summary (sanity check before exporting)
total_customers = len(df)
churn_rate = df["churn_flag"].mean()
avg_monthly = df["MonthlyCharges"].mean()
revenue_at_risk = df["revenue_at_risk_12m"].sum()

print("Total Customers:", total_customers)
print("Churn Rate (%):", round(churn_rate * 100, 2))
print("Avg Monthly Charges:", round(avg_monthly, 2))
print("Revenue at Risk (12m):", round(revenue_at_risk, 2))

Total Customers: 7043
Churn Rate (%): 26.54
Avg Monthly Charges: 64.76
Revenue at Risk (12m): 1669570.2


In [ ]:
# Export Tableau-ready dataset
output_file = "telco_customer_churn_cleaned.csv"
df.to_csv(output_file, index=False)